In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn as sk
from sklearn.model_selection import TimeSeriesSplit

In [22]:
# Read the data that was preprocessed in the EDA step
data = pd.read_csv('input_data_2024-04-07.csv')
data.describe()

,^SOX,^VIX,^VIX^2,^SOX: Log_Returns,^VIX: Log_Returns,^VIX^2: Log_Returns,^SOX: Daily_RV,^SOX: Weekly_RV,^SOX: Daily_RV: Log_Returns,^SOX: Weekly_RV: Log_Returns,epsilon1,epsilon2
count,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000
mean,1370.685336,17.848712,368.417535,0.000691,0.000018,0.000036,0.000329,0.034397,-0.000643,0.000177,0.000510,0.001252
std,972.265520,7.061094,414.852164,0.018128,0.078682,0.157364,0.000950,0.021463,3.169401,0.249976,0.006797,0.223460
min,351.450012,9.140000,83.539606,-0.173119,-0.299831,-0.599662,0.000000,0.003846,-15.944132,-1.344118,-0.028591,-1.151062
25%,616.957520,13.220000,174.768407,-0.008263,-0.044337,-0.088673,0.000018,0.020776,-1.783166,-0.091905,-0.002782,-0.099547
50%,1084.644958,15.930000,253.764910,0.001438,-0.006419,-0.012839,0.000086,0.029321,-0.023617,-0.000573,-0.000760,-0.010562
75%,1847.697540,20.700001,428.490032,0.010198,0.035816,0.071632,0.000307,0.042794,1.822893,0.085001,0.002514,0.089466
max,4039.510010,82.690002,6837.636504,0.105753,0.768245,1.536490,0.029970,0.271085,13.852759,1.793899,0.069019,1.679464


In [23]:
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data

,^SOX,^VIX,^VIX^2,^SOX: Log_Returns,^VIX: Log_Returns,^VIX^2: Log_Returns,^SOX: Daily_RV,^SOX: Weekly_RV,^SOX: Daily_RV: Log_Returns,^SOX: Weekly_RV: Log_Returns,epsilon1,epsilon2
Date,,,,,,,,,,,,
2012-01-09,382.589996,21.070000,443.944887,0.019451,0.021104,0.042208,3.783564e-04,0.024624,3.874993,0.000000,-0.003712,-0.000078
2012-01-10,386.279999,20.690001,428.076122,0.009599,-0.018200,-0.036399,9.213277e-05,0.026428,-1.412606,0.070729,0.001456,0.070650
2012-01-11,387.890015,21.049999,443.102468,0.004159,0.017250,0.034500,1.730011e-05,0.026753,-1.672518,0.012215,-0.000044,0.013082
2012-01-12,391.750000,20.469999,419.020872,0.009902,-0.027940,-0.055880,9.805058e-05,0.024366,1.734771,-0.093473,-0.002835,-0.092581
2012-01-13,383.470001,20.910000,437.228094,-0.021362,0.021267,0.042534,4.563559e-04,0.032283,1.537789,0.281362,0.007507,0.280820
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,2535.489990,20.870001,435.556935,0.000852,-0.051365,-0.102730,7.263040e-07,0.051167,-7.839736,-0.017006,-0.000399,-0.031156
2022-12-27,2490.169922,21.650000,468.722483,-0.018036,0.036693,0.073385,3.252958e-04,0.052457,6.104522,0.024898,0.002141,0.011801
2022-12-28,2453.489990,22.139999,490.179573,-0.014839,0.022380,0.044761,2.202094e-04,0.054155,-0.390156,0.031859,0.002614,-0.006444


In [24]:
# rename the columns (delete special characters)
data = data.rename(columns=lambda x: x.replace('^', '').replace(':', ''))
data.columns

Index(['SOX', 'VIX', 'VIX2', 'SOX Log_Returns', 'VIX Log_Returns',
       'VIX2 Log_Returns', 'SOX Daily_RV', 'SOX Weekly_RV',
       'SOX Daily_RV Log_Returns', 'SOX Weekly_RV Log_Returns', 'epsilon1',
       'epsilon2'],
      dtype='object')

### Data Preprocessing
1. Withormalization
2. Min Max Scaling

In [25]:
# transform numerical data with withormalization
from scipy.stats.mstats import winsorize
data = pd.DataFrame(winsorize(np.array(data), limits=[0.05, 0.05]), columns=data.columns, index=data.index)
data.describe()

,SOX,VIX,VIX2,SOX Log_Returns,VIX Log_Returns,VIX2 Log_Returns,SOX Daily_RV,SOX Weekly_RV,SOX Daily_RV Log_Returns,SOX Weekly_RV Log_Returns,epsilon1,epsilon2
count,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000
mean,756.402765,17.848712,334.242329,0.000691,0.000018,0.001322,0.000329,0.034397,1.027578,0.015640,0.000510,0.010301
std,189.746632,7.061094,219.896506,0.018128,0.078682,0.154199,0.000950,0.021463,1.998203,0.215591,0.006797,0.202905
min,351.450012,9.140000,83.539606,-0.173119,-0.299831,-0.318273,0.000000,0.003846,-0.318273,-0.318273,-0.028591,-0.318273
25%,616.957520,13.220000,174.768407,-0.008263,-0.044337,-0.088673,0.000018,0.020776,-0.318273,-0.091905,-0.002782,-0.099547
50%,907.340027,15.930000,253.764910,0.001438,-0.006419,-0.012839,0.000086,0.029321,-0.023617,-0.000573,-0.000760,-0.010562
75%,907.340027,20.700001,428.490032,0.010198,0.035816,0.071632,0.000307,0.042794,1.822893,0.085001,0.002514,0.089466
max,907.340027,82.690002,907.340027,0.105753,0.768245,1.536490,0.029970,0.271085,13.852759,1.793899,0.069019,1.679464


In [26]:
# transform numerical data with min-max scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
data.describe()

,SOX,VIX,VIX2,SOX Log_Returns,VIX Log_Returns,VIX2 Log_Returns,SOX Daily_RV,SOX Weekly_RV,SOX Daily_RV Log_Returns,SOX Weekly_RV Log_Returns,epsilon1,epsilon2
count,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000,2764.000000
mean,0.728476,0.118405,0.304325,0.623260,0.280737,0.172310,0.010976,0.114322,0.094972,0.158090,0.298137,0.164473
std,0.341338,0.096004,0.266929,0.065003,0.073667,0.083137,0.031686,0.080313,0.141006,0.102071,0.069637,0.101567
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.477626,0.055472,0.110741,0.591154,0.239210,0.123789,0.000606,0.063354,0.000000,0.107173,0.264416,0.109487
50%,1.000000,0.092318,0.206634,0.625939,0.274711,0.164675,0.002854,0.095328,0.020793,0.150414,0.285131,0.154030
75%,1.000000,0.157172,0.418731,0.657351,0.314254,0.210218,0.010252,0.145744,0.151095,0.190929,0.318665,0.204101
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Training and Testing Dataset Preparation
1. Training Data : 2012-2020
2. Testing Data: 2021-2022

In [27]:
# Split the data into training and test sets (training data from 2012 to 2020, test data from 2021 to 2022)
data_train = data.loc['2012-01-01':'2020-12-31']
data_test = data.loc['2021-01-01':'2022-12-31']

### Base Model Attributes(X) and Target(Y)
1. X \
    1.1 Index Value: ^SOX, ^VIX \
    1.2 Square Value: ^VIX^2 \
    1.3 Log Return: ^SOX, ^VIX, ^VIX^2
    
2. y: Weekly Relative Volatility of ^SOX

In [28]:
# Split the data into features and target
X_base_train = data_train[['SOX', 'VIX', 'VIX2', 'SOX Log_Returns', 'VIX Log_Returns', 'VIX2 Log_Returns']]
y_train = data_train[['SOX Weekly_RV']]
X_base_test = data_test[['SOX', 'VIX', 'VIX2', 'SOX Log_Returns', 'VIX Log_Returns', 'VIX2 Log_Returns']]
y_test = data_test[['SOX Weekly_RV']]

### Benchmark Model Training and Performance
1. Decesion Tree Regressor
2. Linear Regression

performance matrix
1. Root Mean Square Error
2. R-square

In [29]:
# Create a decision tree regressor model and use parameters from the grid search
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

model = DecisionTreeRegressor(random_state=42)
model.fit(X_base_train, y_train)

param_grid = {
    'max_depth': range(1, 5),
    'min_samples_split': range(2, 5),
    'min_samples_leaf': range(3, 8),
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=2, n_jobs=-1, verbose=-1)
grid_search.fit(X_base_train, y_train)

# Report the best model parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"Best parameters: {best_params}")
print(f"Best score: {best_score}")

Best parameters: {'max_depth': 4, 'min_samples_leaf': 3, 'min_samples_split': 2}
Best score: 0.1985151583694864


In [30]:
# Predict the target variable for the test set and calculate the RMSE and R2 score
from sklearn.metrics import mean_squared_error, r2_score

model = grid_search.best_estimator_
y_pred = model.predict(X_base_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R2 score: {r2}")

RMSE: 0.06862696849647454
R2 score: 0.15808918073311362


In [31]:
# Create a linear regression model
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_base_train, y_train)

LinearRegression()

In [32]:
# Predict the target variable for the test set and calculate the RMSE and R2 score
y_pred = model.predict(X_base_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R2 score: {r2}")

RMSE: 0.06921342508611589
R2 score: 0.14363848237365262


### LightGBM Model Training and Performance
Hyperparamter tuning: Optuna

Time series split = 5

Objective function: Mean(Root Mean Square Error)

Performance matrix:
1. Root Mean Square Error
2. R-square

Reference:\
https://forecastegy.com/posts/how-to-use-optuna-to-tune-lightgbm-hyperparameters/

In [81]:
# # further split the data into training and validation sets
# data_train = data.loc['2012-01-01':'2019-12-31']
# data_val = data.loc['2020-01-01':'2020-12-31']

# # Split the data into features and target
# X_base_train = data_train[['SOX', 'VIX', 'VIX2', 'SOX Log_Returns', 'VIX Log_Returns', 'VIX2 Log_Returns']]
# y_train = data_train[['SOX Weekly_RV']]
# X_base_val = data_val[['SOX', 'VIX', 'VIX2', 'SOX Log_Returns', 'VIX Log_Returns', 'VIX2 Log_Returns']]
# y_val = data_val[['SOX Weekly_RV']]

In [35]:
tscv = TimeSeriesSplit(n_splits=5)

In [40]:
import lightgbm as lgb
import optuna

def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "n_estimators": 300,
        "verbosity": -1,
        "bagging_freq": 1,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**4),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 20),
    }

    rmse_scores = np.array([])
    model = lgb.LGBMRegressor(**params)

    for train_index, val_index in tscv.split(X_base_train):
        # print("TRAIN:", train_index, "TEST:", val_index)
        X_t, X_val = X_base_train.iloc[train_index], X_base_train.iloc[val_index]
        y_t, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
        model.fit(X_t, y_t)
        predictions = model.predict(X_val)
        rmse = mean_squared_error(y_val, predictions, squared=False)
        rmse_scores = np.append(rmse_scores, rmse)
    return rmse_scores.mean()

In [41]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

[I 2024-04-25 01:30:45,135] A new study created in memory with name: no-name-52bb7e73-9419-432f-b6ad-4e1d6cffe385
[I 2024-04-25 01:30:45,518] Trial 0 finished with value: 0.05951631504644224 and parameters: {'learning_rate': 0.03418273435208915, 'num_leaves': 14, 'subsample': 0.7408923018384463, 'colsample_bytree': 0.8775000733250338, 'min_data_in_leaf': 4}. Best is trial 0 with value: 0.05951631504644224.
[I 2024-04-25 01:30:45,766] Trial 1 finished with value: 0.05700941443864247 and parameters: {'learning_rate': 0.030980360854011683, 'num_leaves': 8, 'subsample': 0.5577591170108473, 'colsample_bytree': 0.7402132309662695, 'min_data_in_leaf': 9}. Best is trial 1 with value: 0.05700941443864247.
[I 2024-04-25 01:30:46,142] Trial 2 finished with value: 0.05787460225063964 and parameters: {'learning_rate': 0.023998305299243163, 'num_leaves': 15, 'subsample': 0.8196687473981048, 'colsample_bytree': 0.6648544021837625, 'min_data_in_leaf': 16}. Best is trial 1 with value: 0.057009414438642

In [46]:
# Calculate the R2 score for the best model
best_params = study.best_params
model = lgb.LGBMRegressor(**best_params)
model.fit(X_base_train, y_train)
# y_pred = model.predict(X_base_val)
# r2 = r2_score(y_val, y_pred)
# print(f"R2 score: {r2}")

[LightGBM] [Warning] min_data_in_leaf is set=3, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=3, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000128 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 2261, number of used features: 6
[LightGBM] [Info] Start training from score 0.102218


LGBMRegressor(colsample_bytree=0.6387891235129918,
              learning_rate=0.028794398546408004, min_data_in_leaf=3,
              num_leaves=2, subsample=0.531292748331675)

In [47]:
# Predict the target variable for the test set and calculate the RMSE and R2 score
y_pred = model.predict(X_base_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R2 score: {r2}")

[LightGBM] [Warning] min_data_in_leaf is set=3, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3
RMSE: 0.07740778285866126
R2 score: -0.0711387172643203


### Feature Engineering
log of yesterday volatility, square of yesterday volatility, log of VIX, log of past 5 days VIX
